In [1]:
# So we can load files from other sub-directories, e.g. datasets.
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../unlabeled_extrapolation'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
from torch import nn
import torchvision.transforms as transforms

import json
import numpy as np
import pickle
from scipy.special import softmax
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import StandardScaler

import calibration as cal
import unlabeled_extrapolation
from unlabeled_extrapolation.models import imnet_resnet

import multiprocessing
print('Number of CPUs:', multiprocessing.cpu_count())

Number of CPUs: 96


In [2]:
print(f'cuda is available: {torch.cuda.is_available()}')
torch.cuda.empty_cache()

cuda is available: True


In [4]:
USE_CUDA = True
cuda = torch.device('cuda')
batch_size, num_workers = 64, 2

In [5]:
# get_breeds dataset (with transform)
from unlabeled_extrapolation.datasets.breeds import Breeds
from unlabeled_extrapolation.datasets.domainnet import DomainNet

breeds_root='/scr/scr-with-most-space/imagenet'
domainnet_root='/scr/scr-with-most-space/domainnet'

breeds_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.228, 0.224, 0.225]
)
breeds_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    breeds_normalize,
])

breeds_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.228, 0.224, 0.225]
)
breeds_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    breeds_normalize,
])

domainnet_normalize = transforms.Normalize(
    mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]
)
domainnet_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    domainnet_normalize,
])

def get_breeds_dataset(breeds_name, source, split):
    dataset = Breeds(
        root=breeds_root, breeds_name=breeds_name, source=source,
        target=not(source), split=split, transform=breeds_transform)
    return dataset
    
def get_breeds_data_loader(breeds_name, source, split, batch_size=64):
    dataset = get_breeds_dataset(breeds_name, source, split)
    loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size,
        shuffle=False, num_workers=num_workers)
    return loader

def get_breeds_uda_loaders(breeds_name, batch_size):
    domains_src = [True, True, False, False]
    splits = ['train', 'val', 'train', 'val']
    num_sets = len(domains_src)
    loaders = []
    for i in range(num_sets):
        cur_loader = get_breeds_data_loader(
            breeds_name=breeds_name, source=domains_src[i],
            split=splits[i], batch_size=batch_size)
        loaders.append(cur_loader)
    return loaders

def get_domainnet_dataset(domain_name, split, version='full'):
    return DomainNet(domain=domain_name, split=split, root=domainnet_root,
                     transform=domainnet_transform, version=version)

def get_domainnet_data_loader(domain_name, split, batch_size=64, version='full'):
    dataset = get_domainnet_dataset(domain_name, split, version=version)
    loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size,
        shuffle=False, num_workers=num_workers)
    return loader

def get_domainnet_uda_loaders(source_name, target_name, batch_size, version='full'):
    domains = [source_name, source_name, target_name, target_name]
    splits = ['train', 'test', 'train', 'test']
    num_sets = len(domains)
    loaders = []
    for i in range(num_sets):
        cur_loader = get_domainnet_data_loader(
            domain_name=domains[i], split=splits[i], batch_size=batch_size, version=version)
        loaders.append(cur_loader)
    return loaders

/u/nlp/anaconda/main/anaconda3/envs/kshen6-uee/lib/python3.8/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `gym` which is not installed yet, install it with `pip install gym`.
  stdout_func(


In [11]:
# Extract features for a model and datasets
def extract_features(model, loader, preprocess=False):
    model.cuda()
    model.eval()
    feature_model = nn.Sequential(*list(model._model.children())[:-1])
    features_list, labels_list = [], []
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images = images.cuda()
            features = feature_model(images)
            features_list.append(features.detach().cpu().numpy())
            labels_list.append(labels.detach().numpy())
    features = np.squeeze(np.concatenate(features_list))
    if preprocess:
        features = StandardScaler().fit_transform(features)
    labels = np.concatenate(labels_list)
    return features, labels

def get_acc(preds, labels):
    return np.mean(preds == labels)

def get_log_reg(features, labels, C=1.0, max_iter=100):
    return LogisticRegression(random_state=0, C=C, max_iter=max_iter, n_jobs=-1).fit(features, labels)

def get_test_accs(features_labels, train_idx, eval_indices, max_iter=100,
                  Cs=[0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 10.0, 100.0]):
    N = len(features_labels)
    clfs = []
    accs = []
    train_features, train_labels = features_labels[train_idx]
    for C in Cs:
        cur_clf = get_log_reg(train_features, train_labels, C=C, max_iter=max_iter)
        clfs.append(cur_clf)
        cur_c_accs = []
        for idx in eval_indices:
            cur_features, cur_labels = features_labels[idx]
            cur_preds = cur_clf.predict(cur_features)
            cur_acc = get_acc(cur_preds, cur_labels)
            cur_c_accs.append(cur_acc)
        print(cur_c_accs)
        accs.append(cur_c_accs)
    return clfs, accs

In [12]:
def run_experiments(
        checkpoint_path,
        loaders,
        pretrain_style='swav',
        preprocess=False,
        max_iter=100,
        source_name=None,
        target_name=None):
    model = imnet_resnet.ResNet50(pretrained=True, pretrain_style=pretrain_style, checkpoint_path=checkpoint_path)
    features_labels_list = []
    for loader in loaders:
        print(loader)
        cur_features, cur_labels = extract_features(model, loader, preprocess=preprocess)
        features_labels_list.append((cur_features, cur_labels))
    print("Train on target, test on target train, target val, source val")
    clfs, accs = get_test_accs(features_labels_list, train_idx=2, eval_indices=[2,3,1], max_iter=max_iter)
    print("Train on source, test on source train, source val, target val")
    clfs, accs = get_test_accs(features_labels_list, train_idx=0, eval_indices=[0,1,3], max_iter=max_iter)
    return loaders, features_labels_list, clfs, accs

def run_breeds_experiments(
        checkpoint_path,
        breeds_name,
        num_classes,
        pretrain_style='swav'):
    loaders = get_breeds_uda_loaders(breeds_name=breeds_name, batch_size=batch_size)
    return run_experiments(checkpoint_path, loaders, pretrain_style)

def run_domainnet_experiments(
        checkpoint_path,
        source_name,
        target_name,
        pretrain_style='swav',
        version='full',
        preprocess=False,
        max_iter=100):
    loaders = get_domainnet_uda_loaders(source_name=source_name, target_name=target_name, batch_size=batch_size,
                                        version=version)
    return run_experiments(checkpoint_path, loaders, pretrain_style, preprocess=preprocess, max_iter=max_iter,
                           scaled_data=scaled_data, source_name=source_name, target_name=target_name)

# DomainNet

In [9]:
DOMAINS = ['clipart', 'painting', 'real', 'sketch']
SPLITS = ['train', 'test']

checkpoint_path = '/u/scr/nlp/eix/ssl_checkpoints/domainnet_swav_200ep_bs256_pretrain/checkpoints/ckp-199.pth'
pretrain_style = 'swav'

model = imnet_resnet.ResNet50(pretrained=True, pretrain_style=pretrain_style, checkpoint_path=checkpoint_path)

# preprocess all the data
if input('Preprocessing data will take a long time. Type \'yes\' to proceed. ') == 'yes':
    for domain in DOMAINS:
        for split in SPLITS:
            filename = f'scaled_data/{domain}-{split}.pickle'
            if os.path.isfile(filename):
                print(f'{domain}/{split} combination has already been calculated. Skipping...')
                continue
            loader = get_domainnet_data_loader(domain, split, batch_size=64, version='full')
            data = extract_features(model, loader, preprocess=True)
            with open(filename, 'wb') as f:
                pickle.dump(data, f)

Preprocessing data will take a long time. Type 'yes' to proceed. yes


In [24]:
# experiments
def load_scaled_data(domain, split):
    with open(f'scaled_data/{domain}-{split}.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

def run_pair(source, target, max_iter=100, Cs=[0.001, 0.01, 0.1, 1.0, 10.0, 100.0]):
    source_train = load_scaled_data(source, 'train')
    source_test = load_scaled_data(source, 'test')
    target_train = load_scaled_data(target, 'train')
    target_test = load_scaled_data(target, 'test')
    data = [source_train, source_test, target_train, target_test]
    print(f'Source {source}, Target {target}')
    print(f'Regularization strengths: {" ".join(list(map(str, Cs)))}')
    print('Train on target, test on target train, target val, source val')
    clfs, accs = get_test_accs(data, train_idx=2, eval_indices=[2,3,1], max_iter=max_iter, Cs=Cs)
    print('Train on source, test on source train, source val, target val')
    clfs, accs = get_test_accs(data, train_idx=0, eval_indices=[0,1,3], max_iter=max_iter, Cs=Cs)

In [26]:
checkpoint_path = '/u/scr/nlp/eix/ssl_checkpoints/domainnet_swav_200ep_bs256_pretrain/checkpoints/ckp-199.pth'
max_iter = 150
source_name = 'real'
target_name = 'painting'
# Cs = [0.1, 1.0, 10.0, 100.0]
run_pair(source_name, target_name, max_iter=max_iter)

Source real, Target painting
Regularization strengths: 0.001 0.01 0.1 1.0 10.0 100.0
Train on target, test on target train, target val, source val
[0.3311448746429705, 0.2751029748283753, 0.15927826137084222]
[0.5508767058076801, 0.32681922196796337, 0.19682557983128687]
[0.8270588701999365, 0.31469107551487413, 0.18343229376837494]
[0.8926927959377975, 0.2820137299771167, 0.1637170692338733]


KeyboardInterrupt: 

In [11]:
checkpoint_path = '/u/scr/nlp/eix/ssl_checkpoints/domainnet_swav_200ep_bs256_pretrain/checkpoints/ckp-199.pth'
source_name = 'sketch'
target_name = 'real'
_, _, _, domainnet_accs = run_domainnet_experiments(checkpoint_path, source_name, target_name, pretrain_style='swav', version='sentry')


/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Train on target, test on target train, target val, source val


/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[0.5195464964995973, 0.5092899323059197, 0.11254689453939141]


/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[0.6245259165613148, 0.4864526886202584, 0.29987037303759184]


/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[0.6292216001444826, 0.47353063776573573, 0.2830188679245283]
[0.6321112515802781, 0.4768653605669029, 0.2824427480916031]


/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Living 17

## Breeds Living 17, swav pre-trained on source + target, 200 epochs

In [26]:
num_classes = 17
batch_size = 64
checkpoint_path = '/u/scr/eix/unlabeled_extrapolation/swav/checkpoints/breeds_living17_source_True_target_True/checkpoints/ckp-199.pth'
breeds_name = 'living17'
# l17_loaders, l17_src_trg_features_labels, l17_src_trg_clfs, l17_src_trg_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)
_, _, _, l17_src_trg_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)

0
1
2
3
Train on target, test on target train, target val, source val


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.8813122171945701, 0.8552941176470589, 0.611764705882353]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.8814932126696833, 0.851764705882353, 0.6094117647058823]


...

[0.8803393665158371, 0.8447058823529412, 0.5870588235294117]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.8797737556561086, 0.8447058823529412, 0.5841176470588235]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.8778959276018099, 0.8452941176470589, 0.5805882352941176]


## Breeds Living 17, swav pre-trained on source + target, 100 epochs (final)

In [7]:
num_classes = 17
batch_size = 64
checkpoint_path = '/u/scr/ananya/cifar_experiments/unlabeled_extrapolation/swav/checkpoints/breeds_living17_source_True_target_True/checkpoints/ckp-99.pth'
breeds_name = 'living17'
# l17_loaders, l17_src_trg_features_labels, l17_src_trg_clfs, l17_src_trg_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)
_, _, _, l17_src_trg_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)

Train on target, test on target train, target val, source val


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.629027149321267, 0.6247058823529412, 0.3776470588235294]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.6593438914027149, 0.6558823529411765, 0.37823529411764706]


...

[0.6843665158371041, 0.638235294117647, 0.4117647058823529]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.6860407239819004, 0.6435294117647059, 0.41058823529411764]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.6852714932126697, 0.6429411764705882, 0.4123529411764706]


## Breeds Living 17, swav pre-trained on source + target, 100 epochs (intermediate)

In [47]:
num_classes = 17
batch_size = 64
checkpoint_path = '/u/scr/eix/unlabeled_extrapolation/swav/checkpoints/breeds_living17_source_True_target_True/checkpoints/ckp-100.pth'
breeds_name = 'living17'
# l17_loaders, l17_src_trg_100_features_labels, l17_src_trg_100_clfs, l17_src_trg_100_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)
_, _, _, l17_src_trg_100_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)

0
1
2
3
Train on target, test on target train, target val, source val


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.7815837104072398, 0.7494117647058823, 0.48823529411764705]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.7825339366515837, 0.7470588235294118, 0.4888235294117647]


...

[0.7757013574660634, 0.7435294117647059, 0.4758823529411765]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.7751809954751131, 0.7441176470588236, 0.4723529411764706]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.7752036199095023, 0.7429411764705882, 0.47705882352941176]


## Breeds Living 17, swav pre-trained on target, 200 epochs (failed run)

In [27]:
num_classes = 17
batch_size = 64
checkpoint_path = '/juice/scr/kshen6/unlabeled_extrapolation/swav/checkpoints/breeds_living17_source_False_target_True/checkpoints/ckp-199.pth'
breeds_name = 'living17'
# l17_loaders, l17_trg_features_labels, l17_trg_clfs, l17_trg_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)
_, _, _, l17_trg_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)

0
1
2
3
Train on target, test on target train, target val, source val
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
Train on source, test on source train, source val, target val
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]


## Breeds Living 17, swav pre-trained on target, 100 epochs (intermediate, failed run)

In [50]:
num_classes = 17
batch_size = 64
checkpoint_path = '/juice/scr/kshen6/unlabeled_extrapolation/swav/checkpoints/breeds_living17_source_False_target_True/checkpoints/ckp-100.pth'
breeds_name = 'living17'
# l17_loaders, l17_trg_100_features_labels, l17_trg_100_clfs, l17_trg_100_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)
_, _, _, l17_trg_100_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)

0
1
2
3
Train on target, test on target train, target val, source val
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
Train on source, test on source train, source val, target val
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705]


## Breeds Living 17, swav pre-trained on all 800 epochs, batch size 4096

In [28]:
num_classes = 17
batch_size = 64
checkpoint_path = '/u/scr/ananya/simclr_weights/swav_800ep_pretrain.pth.tar'
breeds_name = 'living17'
# l17_loaders, l17_all_800_features_labels, l17_all_800_clfs, l17_all_800_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)
_, _, _, l17_all_800_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)

0
1
2
3
Train on target, test on target train, target val, source val


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9897737556561086, 0.9723529411764706, 0.7723529411764706]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9994343891402715, 0.9776470588235294, 0.77]


...

[0.9978054298642534, 0.9629411764705882, 0.808235294117647]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.999524886877828, 0.9594117647058824, 0.8023529411764706]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9996832579185521, 0.9558823529411765, 0.7994117647058824]


## Breeds Living 17, swav pre-trained on all 200 epochs, batch size 256

In [29]:
num_classes = 17
batch_size = 64
checkpoint_path = '/u/scr/ananya/simclr_weights/swav_200ep_bs256_pretrain.pth.tar'
breeds_name = 'living17'
# l17_loaders, l17_all_200_features_labels, l17_all_200_clfs, l17_all_200_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)
_, _, _, l17_all_200_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)

0
1
2
3
Train on target, test on target train, target val, source val


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9863122171945702, 0.9664705882352941, 0.7711764705882352]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9985972850678733, 0.9664705882352941, 0.7688235294117647]


...

[0.9960407239819005, 0.961764705882353, 0.8023529411764706]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9984841628959276, 0.96, 0.7852941176470588]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9992760180995475, 0.9558823529411765, 0.7811764705882352]


## Breeds Living 17, moco pre-trained on all 800 epochs, batch size 256 

In [49]:
num_classes = 17
batch_size = 64
checkpoint_path = '/u/scr/ananya/simclr_weights/moco_v2_800ep_pretrain.pth.tar'
breeds_name = 'living17'
# l17_loaders, l17_all_moco_features_labels, l17_all_moco_clfs, l17_all_moco_accs = run_breeds_experiments(
#     checkpoint_path, breeds_name, num_classes, pretrain_style='mocov2')
_, _, _, l17_all_moco_accs = run_breeds_experiments(
    checkpoint_path, breeds_name, num_classes, pretrain_style='mocov2')

0
1
2
3
Train on target, test on target train, target val, source val
[0.9616063348416289, 0.9488235294117647, 0.8323529411764706]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9724434389140272, 0.9611764705882353, 0.8352941176470589]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9856108597285068, 0.9647058823529412, 0.8294117647058824]


...

[0.9672398190045249, 0.9588235294117647, 0.8129411764705883]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9820135746606334, 0.9635294117647059, 0.8164705882352942]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.988710407239819, 0.9570588235294117, 0.8076470588235294]


# Entity 30

## Entity 30, swav target-only, 100 epochs (intermediate)

In [ ]:
num_classes = 30
batch_size = 64
checkpoint_path = '/u/scr/ananya/cifar_experiments/unlabeled_extrapolation/swav/checkpoints/breeds_entity30_source_False_target_True/checkpoints/ckp-100.pth'
breeds_name = 'entity30'
# e30_loaders, e30_trg_100_features_labels, e30_trg_100_clfs, e30_trg_100_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)
_, _, _, e30_trg_100_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)

## Entity 30, swav source + target, 100 epochs (intermediate, failed run)

In [10]:
num_classes = 17
batch_size = 64
checkpoint_path = '/juice/scr/rmjones/unlabeled_extrapolation/swav/checkpoints/breeds_entity30_source_True_target_True/checkpoints/ckp-100.pth'
breeds_name = 'entity30'
# e30_loaders, e30_src_trg_100_features_labels, e30_src_trg_100_clfs, e30_src_trg_100_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)
_, _, _, e30_src_trg_100_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)

Train on target, test on target train, target val, source val
[0.0392407124019145, 0.0415, 0.03716666666666667]
[0.039032331585973364, 0.041666666666666664, 0.036833333333333336]
[0.03894767687949728, 0.04133333333333333, 0.036833333333333336]
[0.03892162927750464, 0.041166666666666664, 0.036833333333333336]
[0.03891511737700648, 0.041166666666666664, 0.03666666666666667]
[0.03890860547650832, 0.041166666666666664, 0.03666666666666667]
[0.03890860547650832, 0.041166666666666664, 0.03666666666666667]
Train on source, test on source train, source val, target val
[0.03849270401846198, 0.03866666666666667, 0.0385]
[0.03835009042997997, 0.03866666666666667, 0.0385]
[0.03827230119989888, 0.0385, 0.03866666666666667]
[0.03827230119989888, 0.03866666666666667, 0.0385]
[0.03824637145653851, 0.03883333333333333, 0.0385]
[0.038239889020698414, 0.03883333333333333, 0.0385]
[0.03824637145653851, 0.03883333333333333, 0.0385]


## Entity 30, swav target-only, 200 epochs

In [13]:
num_classes = 30
batch_size = 64
checkpoint_path = '/u/scr/ananya/cifar_experiments/unlabeled_extrapolation/swav/checkpoints/breeds_entity30_source_False_target_True/checkpoints/ckp-199.pth'
breeds_name = 'entity30'
# e30_loaders, e30_trg_features_labels, e30_trg_clfs, e30_trg_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)
_, _, _, e30_trg_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)

OSError: [Errno 12] Cannot allocate memory

## Entity 30, swav source + target, 200 epochs (failed run)

In [11]:
num_classes = 17
batch_size = 64
checkpoint_path = '/juice/scr/rmjones/unlabeled_extrapolation/swav/checkpoints/breeds_entity30_source_True_target_True/checkpoints/ckp-199.pth'
breeds_name = 'entity30'
# e30_loaders, e30_src_trg_features_labels, e30_src_trg_clfs, e30_src_trg_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)
_, _, _, e30_src_trg_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)

Train on target, test on target train, target val, source val
[0.03960537882981148, 0.042, 0.037333333333333336]
[0.03959886692931332, 0.042, 0.037333333333333336]
[0.03960537882981148, 0.042, 0.037333333333333336]
[0.03960537882981148, 0.042, 0.037333333333333336]
Train on source, test on source train, source val, target val
[0.039199289525031926, 0.03966666666666667, 0.03966666666666667]
[0.039179842217511654, 0.03966666666666667, 0.03966666666666667]
[0.039179842217511654, 0.03966666666666667, 0.03966666666666667]
[0.039179842217511654, 0.03966666666666667, 0.03966666666666667]


## Entity 30, swav pre-trained on all 800 epochs, batch size 4096

In [8]:
num_classes = 30
batch_size = 64
checkpoint_path = '/u/scr/ananya/simclr_weights/swav_800ep_pretrain.pth.tar'
breeds_name = 'entity30'
# e30_loaders, e30_all_800_features_labels, e30_all_800_clfs, e30_all_800_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)
_, _, _, e30_all_800_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)

Train on target, test on target train, target val, source val


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.8766646045648423, 0.8601666666666666, 0.6466666666666666]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9014423859603425, 0.8871666666666667, 0.6583333333333333]


...

[0.9629852913530789, 0.9206666666666666, 0.6168333333333333]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9643660501870183, 0.9183333333333333, 0.6065]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9642558487777367, 0.9196666666666666, 0.607]


## Entity 30, swav pre-trained on all 200 epochs, batch size 256

In [8]:
num_classes = 30
batch_size = 64
checkpoint_path = '/u/scr/ananya/simclr_weights/swav_200ep_bs256_pretrain.pth.tar'
breeds_name = 'entity30'
# e30_loaders, e30_all_200_features_labels, e30_all_200_clfs, e30_all_200_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)
_, _, _, e30_all_200_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes)

Train on target, test on target train, target val, source val


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9101943802298701, 0.8921666666666667, 0.6695]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.926838797903168, 0.9043333333333333, 0.6695]


...

/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9394864614327479, 0.9048333333333334, 0.6228333333333333]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9399402319415544, 0.9055, 0.621]


([<torch.utils.data.dataloader.DataLoader at 0x7fb2a0440550>,
 [(array([[0.02918463, 0.14709285, 0.00786341, ..., 0.        , 0.00553668,
           0.        ],
          [0.05966679, 0.20911536, 0.2825517 , ..., 0.10157217, 0.0087694 ,
           0.        ],
          [0.49824154, 0.20402619, 0.19001405, ..., 0.        , 0.        ,
           0.05795331],
          ...,
          [0.2822736 , 0.03942383, 0.00122997, ..., 0.01295149, 0.        ,
           0.23418796],
          [0.46481586, 0.12833206, 0.        , ..., 0.        , 0.08500823,
           0.01919867],
          [0.14513768, 0.02540708, 0.04396388, ..., 0.02075141, 0.        ,
           0.03360466]], dtype=float32),
   array([ 0,  0,  0, ..., 29, 29, 29])),
  (array([[1.3398409e-01, 1.1947680e-01, 1.2321285e-01, ..., 0.0000000e+00,
           3.7387961e-03, 7.0076208e-03],
          [6.5605782e-02, 2.8351992e-01, 3.5790104e-01, ..., 2.1075816e-01,
           5.2589655e-02, 1.2945886e-04],
          [1.6202983e-01, 2.

## Entity 30, moco pre-trained on all 800 epochs

In [9]:
num_classes = 30
batch_size = 64
checkpoint_path = '/u/scr/ananya/simclr_weights/moco_v2_800ep_pretrain.pth.tar'
breeds_name = 'entity30'
# e30_loaders, e30_all_moco_features_labels, e30_all_moco_clfs, e30_all_moco_accs = run_breeds_experiments(
#     checkpoint_path, breeds_name, num_classes, pretrain_style='mocov2')
_, _, _, e30_all_moco_accs = run_breeds_experiments(checkpoint_path, breeds_name, num_classes, pretrain_style='mocov2')

Train on target, test on target train, target val, source val
[0.8497834793084361, 0.8391666666666666, 0.6785]
[0.8678279555888386, 0.8548333333333333, 0.6878333333333333]
[0.8870055025559209, 0.8773333333333333, 0.6933333333333334]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9017744928857487, 0.8888333333333334, 0.6935]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9163481262006317, 0.9, 0.685]


...

/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9253871634805495, 0.9056666666666666, 0.652]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.9274226483343381, 0.9043333333333333, 0.6313333333333333]


([<torch.utils.data.dataloader.DataLoader at 0x7fb31dfd6a10>,
 [(array([[0.        , 0.        , 0.        , ..., 0.00089777, 0.02390817,
           0.00403116],
          [0.00394008, 0.00158987, 0.00047079, ..., 0.00312029, 0.        ,
           0.        ],
          [0.00110936, 0.00103289, 0.        , ..., 0.00984201, 0.02026106,
           0.        ],
          ...,
          [0.00536309, 0.00042262, 0.        , ..., 0.00156597, 0.00079828,
           0.03773031],
          [0.00966642, 0.00136781, 0.        , ..., 0.00026091, 0.        ,
           0.00318735],
          [0.00602908, 0.        , 0.        , ..., 0.00237962, 0.00273056,
           0.00103408]], dtype=float32),
   array([ 0,  0,  0, ..., 29, 29, 29])),
  (array([[0.0000000e+00, 0.0000000e+00, 1.5270246e-04, ..., 2.9348186e-03,
           1.8669110e-02, 2.6641131e-04],
          [3.4338306e-03, 1.4282480e-03, 0.0000000e+00, ..., 7.3860167e-05,
           0.0000000e+00, 0.0000000e+00],
          [3.0524286e-03, 0.

# Older runs

In [51]:
print(np.mean(l17_trg_features_labels[2][0]))
print(np.mean(l17_src_trg_features_labels[2][0]))
print(np.mean(l17_all_200_features_labels[2][0]))
print(np.mean(l17_all_800_features_labels[2][0]))

3.0187484e-05
0.25676548
0.07863089
0.08567586


In [20]:
# Train on target, evaluate on target train, target val, source val
clfs, accs = get_test_accs(l17_src_trg_features_labels_list, train_idx=2, eval_indices=[2,3,1])

/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.8813122171945701, 0.8552941176470589, 0.611764705882353]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.8814932126696833, 0.851764705882353, 0.6094117647058823]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.8816968325791855, 0.85, 0.6052941176470589]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.8814705882352941, 0.8476470588235294, 0.6123529411764705]


In [21]:
# Train on source, evaluate on source train, source val, target val
clfs, accs = get_test_accs(l17_src_trg_features_labels_list, train_idx=0, eval_indices=[0,1,3])

/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.878868778280543, 0.8429411764705882, 0.5858823529411765]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.8803393665158371, 0.8447058823529412, 0.5870588235294117]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.8797737556561086, 0.8447058823529412, 0.5841176470588235]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.8778959276018099, 0.8452941176470589, 0.5805882352941176]


In [ ]:
# Breeds Living 17, swav pre-trained on source + target
num_classes = 17
batch_size = 64
checkpoint_path = '/u/scr/eix/unlabeled_extrapolation/swav/checkpoints/breeds_living17_source_True_target_True/checkpoints/ckp-199.pth'
breeds_name = 'living17'


l17_loaders = get_breeds_uda_loaders(breeds_name=breeds_name, batch_size=batch_size)
l17_trg_model = imnet_resnet.ResNet50(pretrained=True, pretrain_style='swav', checkpoint_path=checkpoint_path)

l17_trg_features_labels_list = []
for loader in loaders:
    print(loader)
    cur_features, cur_labels = extract_features(l17_trg_model, l17_loaders)
    l17_trg_features_labels_list.append((cur_features, cur_labels))
